In [1]:
!pip install adapters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 117.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [2]:
# @title imports
import pandas as pd
import torch 
from adapters import AutoAdapterModel
from transformers import AutoTokenizer,DataCollatorWithPadding
from tqdm import tqdm

2025-12-15 05:57:00.979152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765778221.156353      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765778221.203843      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [27]:
# @title global Variables
df_itr=pd.read_json("/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json",lines=True,chunksize=10000)
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
# @title loading models and tokenizer
ad_hoc=AutoAdapterModel.from_pretrained("allenai/specter2_base")
ad_hoc.load_adapter("allenai/specter2_adhoc_query", source="hf", load_as="specter2_adhoc_query", set_active=True)
tokenizer=AutoTokenizer.from_pretrained("allenai/specter2_base")

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.59M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
ad_hoc(**tokenizer('Model for transformers in vision. good one',return_tensors="pt",return_token_type_ids=False)).last_hidden_state[:,0,:].shape

torch.Size([1, 768])

In [6]:
ad_hoc.eval()
ad_hoc=ad_hoc.to(device)

In [7]:
import os
import numpy as np
os.makedirs("/kaggle/working/chunks",exist_ok=True)

In [8]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self,df,tokenizer):
        self.df=df[['title','abstract','id']]
        self.df['input']=df['title']+tokenizer.sep_token+df['abstract']
        self.df.drop(columns=['abstract'],inplace=True)
        self.tokenizer=tokenizer
        
    def __len__(self):
        return self.df.shape[0]
        
    def __getitem__(self,idx):
        inp=self.df["input"].values[idx]
        return {"id":self.df['id'].values[idx],"input":inp,"title":self.df['title'].values[idx]}

In [9]:
def customCollator(batch):
    inp=[item['input'] for item in batch]
    inp=tokenizer(inp,return_tensors='pt',padding=True,truncation=True,return_token_type_ids=False,max_length=512)

    return {
        "id":[item['id'] for item in batch],
        "title":[item['title'] for item in batch],
        "input": inp,
    }

In [10]:
import warnings
warnings.simplefilter("ignore")

In [28]:
for i,df in tqdm(enumerate(df_itr)):
    if(i==4):
        break
    data_loader=torch.utils.data.DataLoader(customDataset(df,tokenizer),collate_fn=customCollator,batch_size=256)
    dct={"id":[],"embed":[],"title":[]}
    for x in tqdm(data_loader):
        inp={k:v.to(device) for k,v in x['input'].items()}
        with torch.no_grad():
            out=ad_hoc(**inp).last_hidden_state[:,0,:]
        # print(out.shape)
        dct['id']=dct['id']+x['id']
        dct['title']=dct['title']+x['title']
        dct['embed']=dct['embed']+list(out.to('cpu').detach().numpy())
        # print(dct)
        # break
        del inp,out
        torch.cuda.empty_cache()
    pd.DataFrame(dct).to_csv(f"/kaggle/working/chunks/{i}.csv",index=False)
    # break

0it [00:00, ?it/s]
100%|██████████| 40/40 [02:43<00:00,  4.09s/it]
1it [03:22, 202.76s/it]
100%|██████████| 40/40 [02:42<00:00,  4.07s/it]
2it [06:45, 202.70s/it]
100%|██████████| 40/40 [02:43<00:00,  4.08s/it]
3it [10:07, 202.57s/it]
100%|██████████| 40/40 [02:42<00:00,  4.07s/it]
4it [13:31, 202.94s/it]


In [12]:
new_df=pd.read_csv("/kaggle/working/chunks/0.csv")
np.fromstring(new_df['embed'].values[0].strip("[]"),sep=" ").shape

(768,)

In [36]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [37]:
# code to combine all of chunks files
import os
main_df=None
for file in os.listdir("/kaggle/working/chunks"):
    if main_df is None:
        main_df=pd.read_csv(f"/kaggle/working/chunks/{file}")
    else:
        df=pd.read_csv(f"/kaggle/working/chunks/{file}")
        main_df=pd.concat((main_df,df),axis=0)
main_df.to_csv("/kaggle/working/final_file.csv",index=False)
combined_csv_path="/kaggle/working/final_file.csv"

In [13]:
!pip install faiss-gpu
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 85.5 MB/s eta 0:00:00:00:0100:01


In [38]:
import faiss

nlist = 1024  # number of cells/clusters to partition data into
index=faiss.IndexFlatIP(768)
index=faiss.IndexIVFFlat(index,768,nlist)
df=pd.read_csv(combined_csv_path)
embeddings=np.array([np.fromstring(i.strip("[]"),sep=" ") for i in df['embed'].values])
index.train(embeddings)
index.add(embeddings)

In [39]:
# generally used value
# nlist ≈ √df.shape[1]
# nprobe ≈ √nlist
nprobe=64
index.nprobe=nprobe

In [57]:
# checking index
tokens=tokenizer('transformers in Deep Learning',return_tensors="pt",return_token_type_ids=False)
xq=ad_hoc(**{k:v.to(device) for k,v in tokens.items()}).last_hidden_state[:,0,:]
k=5
D, I = index.search(xq.to('cpu').detach().numpy(), k)

In [58]:
df.iloc[I[0],:]['title'].values

array(['Topological phase transition in complex networks',
       'Complex Networks in and beyond Physics',
       'Normalisation Control in Deep Inference via Atomic Flows',
       'Resource modalities in game semantics',
       'Optimization in Gradient Networks'], dtype=object)

In [63]:
df.drop(columns=['embed'],inplace=True)
df.to_csv(combined_csv_path,index=False)

In [64]:
# saving index
faiss.write_index(index, "/kaggle/working/index.faiss")

In [ ]:
import boto3
import os

# setting env vars

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = ""

In [61]:
bucket="research-paper-search-engine"
s3 = boto3.client("s3")
def upload_file_to_s3(file_path,bucket,object_name):
    s3.upload_file(file_path,bucket,object_name)

In [67]:
upload_file_to_s3("/kaggle/working/final_file.csv",bucket,"title_csv_file.csv")

In [68]:
upload_file_to_s3("/kaggle/working/index.faiss",bucket,"index.faiss")